In [27]:
import scipy as cp
import sklearn
import spacy

import sense2vec

In [2]:
en_nlp = spacy.load('en')

In [3]:
# TODO
# Learn a little spacy and sense2vec
# Make a function to build an inverted index and a tfidf vectorizer given tokenized text.

In [ ]:
# IMPORTANT!

# Part of speech tags
# ADJ ADP ADV AUX CONJ DET INTJ NOUN NUM PART PRON PROPN PUNCT SCONJ SYM VERB X

# Named entities tags
# NORP FACILITY ORG GPE LOC PRODUCT EVENT WORK_OF_ART LANGUAGE

In [17]:
# Loading model
model_s2v = sense2vec.load('reddit_vectors')

([u'friend|NOUN',
  u'other_friend|NOUN',
  u'brother|NOUN',
  u'good_friend|NOUN',
  u'freind|NOUN'],
 <MemoryView of 'ndarray' at 0x7fe44bafa590>)

In [ ]:
# Finding n most similar word vectors to a particular word vector
freq, query_vector = model_s2v[u'friend|NOUN']
model_s2v.most_similar(query_vector, n =5)

In [32]:
# Finding how similar two word vectors are each other on a 0 to 1 scale
freq1, vector1 = model_s2v[u'foil|NOUN']
freq2, vector2 = model_s2v[u'foil|VERB']
print model_s2v.data.similarity(vector1, vector2)

0.673017501831
